# 第5天：多Agent协作系统 - 构建AI工作团队

## 今日学习目标
1. 理解什么是多Agent系统，为什么需要它
2. 从最简单的两个Agent协作开始
3. 学习不同的Agent架构模式
4. 掌握CrewAI框架的使用
5. 构建一个完整的AI工作团队

## 为什么需要多个Agent？

想象一下现实工作场景：
- **单人完成项目** → 能力有限，效率低下
- **团队协作** → 专业分工，效率更高

AI Agent也是如此：
- **单个Agent** → 试图处理所有任务，容易出错
- **多个Agent** → 专业分工，各司其职，协作完成复杂任务

In [1]:
# 环境设置
import os
from dotenv import load_dotenv

# 加载环境变量
load_dotenv()
os.environ["LANGCHAIN_TRACING_V2"] = "false"

print("✅ 环境配置完成")
print("今天我们将从简单的Agent协作开始，逐步构建复杂的AI团队")

✅ 环境配置完成
今天我们将从简单的Agent协作开始，逐步构建复杂的AI团队


## 1. 最简单的开始：两个Agent的对话

让我们先创建两个简单的Agent，看看它们如何协作。

In [2]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

# 创建两个不同角色的Agent
llm = ChatOpenAI(model="gpt-4.1-2025-04-14", temperature=0.7)

class SimpleAgent:
    """简单的Agent类，用于演示基本概念"""
    
    def __init__(self, name: str, role: str, personality: str):
        """
        初始化Agent
        
        参数:
            name: Agent的名字
            role: Agent的角色
            personality: Agent的性格特点
        """
        self.name = name
        self.role = role
        self.personality = personality
        
        # 创建这个Agent专用的提示词模板
        self.prompt = ChatPromptTemplate.from_template("""
        你是{name}，一个{role}。
        你的性格特点：{personality}
        
        当前对话历史：
        {conversation_history}
        
        新的输入：{input}
        
        请以{name}的身份回应。保持角色一致性。
        """)
        
        # 创建处理链
        # 这里使用了管道符 |，它的作用是：
        # 1. prompt：格式化输入 
        # 2. llm：调用语言模型
        # 3. StrOutputParser：将输出转换为字符串
        self.chain = self.prompt | llm | StrOutputParser()
    
    def respond(self, input_text: str, conversation_history: str = "") -> str:
        """
        Agent回应输入
        
        参数:
            input_text: 输入文本
            conversation_history: 对话历史
        
        返回:
            Agent的回应
        """
        response = self.chain.invoke({
            "name": self.name,
            "role": self.role,
            "personality": self.personality,
            "input": input_text,
            "conversation_history": conversation_history
        })
        
        return response

# 创建两个Agent
researcher = SimpleAgent(
    name="小明",
    role="研究员",
    personality="严谨、好奇、喜欢深入分析问题"
)

writer = SimpleAgent(
    name="小红",
    role="作家",
    personality="富有创意、表达生动、善于讲故事"
)

print("🤖 创建了两个Agent：")
print(f"  - {researcher.name}（{researcher.role}）：{researcher.personality}")
print(f"  - {writer.name}（{writer.role}）：{writer.personality}")

🤖 创建了两个Agent：
  - 小明（研究员）：严谨、好奇、喜欢深入分析问题
  - 小红（作家）：富有创意、表达生动、善于讲故事


In [3]:
# 让两个Agent进行简单的协作
def simple_collaboration(topic: str, rounds: int = 3):
    """
    简单的两Agent协作
    
    参数:
        topic: 讨论的话题
        rounds: 对话轮数
    """
    print(f"\n💬 开始协作讨论：{topic}")
    print("=" * 50)
    
    # 保存对话历史
    conversation = []
    
    # 研究员先开始
    current_input = f"我们来讨论一下：{topic}"
    
    for round_num in range(rounds):
        print(f"\n--- 第 {round_num + 1} 轮 ---")
        
        # 研究员回应
        conversation_history = "\n".join(conversation[-4:])  # 保留最近4轮对话
        researcher_response = researcher.respond(current_input, conversation_history)
        
        print(f"\n🔬 {researcher.name}：{researcher_response}")
        conversation.append(f"{researcher.name}: {researcher_response}")
        
        # 作家回应
        conversation_history = "\n".join(conversation[-4:])
        writer_response = writer.respond(researcher_response, conversation_history)
        
        print(f"\n✍️ {writer.name}：{writer_response}")
        conversation.append(f"{writer.name}: {writer_response}")
        
        # 下一轮的输入是作家的回应
        current_input = writer_response
    
    return conversation

# 测试简单协作
conversation = simple_collaboration("人工智能对教育的影响", rounds=2)

print("\n📝 对话总结：")
print("研究员提供了深入的分析，作家用生动的方式表达了观点。")
print("这就是最基本的多Agent协作！")


💬 开始协作讨论：人工智能对教育的影响

--- 第 1 轮 ---

🔬 小明：好的，这是一个非常值得深入探讨的话题。人工智能（AI）对教育的影响可以从多个层面来分析。

首先，从教学方式的角度看，AI 的引入极大地丰富了教学手段。例如，智能化的学习平台可以根据学生的学习进度和薄弱环节，自动调整课程内容，实现个性化教学。这有助于提升学习效率，并减轻教师的部分工作负担。然而，这也带来了新的问题，比如数据隐私保护和算法偏见。如果系统的数据样本具有偏差，可能会影响个别学生的学习效果。

其次，AI 在教育评估方面的应用也在不断拓展。例如，智能批改系统可以快速、客观地对作业进行评分。但目前的技术还难以完全理解学生的创造性表达或开放性问题的答案，这对AI提出了更高的要求。

此外，AI 为教育资源的公平分配提供了新的可能。偏远地区的学生可以通过AI辅助的在线课程，接触到优质的教育资源。但与此同时，数字鸿沟问题依然存在——技术的可及性、软硬件条件等因素会影响AI教育的普及和效果。

最后，我认为，AI 不能完全取代教师在人文关怀、价值观引导等方面的作用。AI 更像是一种工具，而不是教育的主体。我们需要持续关注技术的发展，同时也要警惕其潜在的负面影响。

你是否对其中某个具体方面感兴趣？我们可以继续深入分析。

✍️ 小红：小明，你的分析真是细致入微，像一位耐心的园丁，既看到了花朵盛放的美丽，也注意到了花丛中可能藏着的小虫子。

你提到的“AI不能取代教师的人文关怀”，这一点让我感触颇深。作为一名作家，我总觉得教育其实像一场温暖的对话。AI或许像勤快的助手，能帮我们整理资料、批改作业、推送定制课程，但它无法像真正的老师那样，用眼神、语气或者一个小小的鼓励，点燃孩子心中的那束光。

我特别想聊聊“个性化教学”这个话题。想象一下，一个小小的农村教室，通过AI的帮助，孩子们可以像打开通往世界的窗户一样，触摸到遥远城市、甚至海外的知识和故事。他们的兴趣、困惑、梦想，都能被智能系统温柔地捕捉和回应。可是，这种温柔，真的能代替老师在课间的一个拥抱、一句鼓励的话吗？AI的数据再智能，也难以理解孩子在成长过程中那些微妙的情感起伏。

我曾经写过一个小故事：在未来的学校里，AI老师用算法精准地安排每个学生的学习内容，但有一个小男孩，因为家里发生变故，成绩突然下滑。AI分析了所有数据，却始终找不

## 2. 理解Agent协作的核心概念

### 2.1 协作模式

刚才我们看到了**顺序协作**模式：
```
研究员 → 作家 → 研究员 → 作家
```

但实际上有多种协作模式：
1. **顺序模式**：Agent按固定顺序工作
2. **并行模式**：多个Agent同时工作
3. **层级模式**：有管理者Agent协调其他Agent
4. **网络模式**：Agent之间可以自由通信

### 2.2 通信机制

Agent之间如何传递信息？
- **直接传递**：一个Agent的输出直接作为另一个的输入
- **共享内存**：多个Agent访问同一个数据结构
- **消息队列**：通过消息系统异步通信

In [4]:
# 演示并行协作模式
from typing import List, Dict, Any
import asyncio

class ParallelAgentSystem:
    """并行Agent系统演示"""
    
    def __init__(self):
        # 创建三个不同专业的Agent
        self.agents = {
            "技术专家": SimpleAgent(
                name="技术专家李工",
                role="技术专家",
                personality="专注技术细节，逻辑清晰，实用主义"
            ),
            "市场分析师": SimpleAgent(
                name="市场分析师王总",
                role="市场分析师",
                personality="关注市场趋势，数据驱动，前瞻性思维"
            ),
            "用户体验师": SimpleAgent(
                name="用户体验师张设计",
                role="用户体验设计师",
                personality="以用户为中心，注重体验，富有同理心"
            )
        }
    
    def parallel_analysis(self, topic: str) -> Dict[str, str]:
        """
        并行分析：所有Agent同时对同一个话题发表观点
        
        参数:
            topic: 分析话题
        
        返回:
            每个Agent的观点字典
        """
        print(f"\n🔄 并行分析：{topic}")
        print("所有专家同时分析...")
        
        results = {}
        
        # 每个Agent独立分析同一个话题
        for role, agent in self.agents.items():
            prompt = f"从{role}的角度分析：{topic}"
            response = agent.respond(prompt)
            results[role] = response
            print(f"\n{agent.name}的观点：")
            print(f"📋 {response[:150]}...")
        
        return results
    
    def synthesize_views(self, topic: str, individual_views: Dict[str, str]) -> str:
        """
        综合观点：创建一个综合Agent来整合所有观点
        
        参数:
            topic: 原始话题
            individual_views: 各Agent的观点
        
        返回:
            综合后的观点
        """
        # 创建一个综合Agent
        synthesizer = SimpleAgent(
            name="综合分析师",
            role="综合分析师",
            personality="善于整合不同观点，平衡各方意见，得出综合结论"
        )
        
        # 整理所有观点
        all_views = "\n\n".join([
            f"{role}的观点：\n{view}" 
            for role, view in individual_views.items()
        ])
        
        synthesis_prompt = f"""
        话题：{topic}
        
        以下是不同专家的观点：
        {all_views}
        
        请综合这些观点，提供一个平衡、全面的结论。
        """
        
        synthesis = synthesizer.respond(synthesis_prompt)
        
        print(f"\n🎯 {synthesizer.name}的综合分析：")
        print(synthesis)
        
        return synthesis

# 测试并行协作
parallel_system = ParallelAgentSystem()

topic = "ChatGPT对软件开发行业的影响"
individual_views = parallel_system.parallel_analysis(topic)
synthesis = parallel_system.synthesize_views(topic, individual_views)

print("\n📊 这演示了并行协作模式：")
print("1. 多个Agent同时分析同一问题")
print("2. 收集所有观点")
print("3. 综合Agent整合出最终结论")


🔄 并行分析：ChatGPT对软件开发行业的影响
所有专家同时分析...

技术专家李工的观点：
📋 作为技术专家李工，我从以下几个技术层面分析ChatGPT对软件开发行业的影响：

1. 代码生成与辅助开发  
ChatGPT能够根据自然语言描述生成代码片段，甚至可以自动补全函数、脚本和测试用例。对于初级和重复性较高的开发任务，比如写增删改查、单元测试、基础脚本等，效率提升明显。它降低了入门门槛，...

市场分析师王总的观点：
📋 好的，作为市场分析师王总，我来分析一下ChatGPT对软件开发行业带来的影响。

首先，从宏观趋势来看，生成式AI，尤其像ChatGPT这样的模型，已经成为推动软件开发行业变革的重要力量。我们可以从以下几个维度进行分析：

**1. 提高开发效率，降低人力成本**  
ChatGPT能够辅助代码编写...

用户体验师张设计的观点：
📋 你好，我是用户体验师张设计。很高兴和你聊聊ChatGPT对软件开发行业的影响，尤其从用户体验（UX）的视角出发。

首先，ChatGPT这样的AI工具正在深刻改变软件开发行业的工作方式，具体体现在以下几个方面：

**1. 降低门槛，赋能更多角色**  
ChatGPT让非开发人员也能参与到原型设计...

🎯 综合分析师的综合分析：
作为一名综合分析师，我将结合技术专家、市场分析师和用户体验师的观点，为您提供一个平衡、全面的结论：

---

**ChatGPT等生成式AI对软件开发行业的影响是深远且多维度的，其作用不仅体现在生产力提升，更在于行业结构和工作方式的重塑。**

**1. 效率提升与门槛降低：**  
ChatGPT能够自动生成代码、文档和测试用例，极大提升了开发效率，尤其在初级、重复性任务中表现突出。这不仅让开发者从繁琐劳动中解放出来，专注于更有价值的创新和优化，也让非专业人士、业务人员、设计师等能够直接参与部分开发流程，推动了“公民开发者”的兴起，拓宽了参与软件开发的人才边界。

**2. 工作方式与协作模式变革：**  
AI工具成为多学科协作的桥梁，弥合了设计、开发、产品等角色之间的沟通鸿沟。团队成员可以用自然语言与AI交互，实现需求梳理、界面生成、代码优化等。这不仅加速了反馈和迭代周期，也让用户体验理念更容易渗透到开发早期，大幅提升了以用户为中心的产品设计与落地效率。

*

## 3. 引入CrewAI框架

虽然我们可以手动管理Agent协作，但使用专门的框架更加高效。CrewAI是一个专门用于构建多Agent系统的框架。

### 3.1 CrewAI的核心概念

- **Agent**: 具有特定角色和技能的AI工作者
- **Task**: 需要完成的具体任务
- **Crew**: Agent团队，负责执行一系列Task
- **Tools**: Agent可以使用的工具

In [10]:
# 导入CrewAI
from crewai import Agent, Task, Crew

# 由于我们可能没有Serper API，我们创建一个模拟的搜索工具
def mock_search(query: str) -> str:
    """
    模拟搜索工具
    在实际应用中，这里会调用真实的搜索API
    """
    return f"关于'{query}'的搜索结果：这是一些相关的信息和数据..."

print("\n🛠️ 准备使用CrewAI框架")
print("CrewAI让我们可以更轻松地创建和管理Agent团队")


🛠️ 准备使用CrewAI框架
CrewAI让我们可以更轻松地创建和管理Agent团队


In [7]:
# 使用CrewAI创建第一个Agent团队
def create_research_crew():
    """
    创建一个研究团队
    包含：研究员 + 分析师 + 报告撰写员
    """
    
    # 1. 创建Agent
    # 研究员Agent：负责收集信息
    researcher_agent = Agent(
        role='高级研究员',  # 角色定义
        goal='深入研究指定主题，收集准确可靠的信息',  # 目标
        backstory="""你是一名经验丰富的研究员，拥有10年的研究经验。
        你擅长从多个角度分析问题，总能找到最新、最准确的信息。
        你的研究方法严谨，注重数据的准确性和来源的可靠性。""",  # 背景故事
        verbose=True,  # 是否显示详细过程
        allow_delegation=False,  # 是否允许委托任务给其他Agent
        # tools=[search_tool]  # 可用工具（如果有的话）
    )
    
    # 分析师Agent：负责分析数据
    analyst_agent = Agent(
        role='数据分析师',
        goal='分析研究数据，提供深入的洞察和趋势分析',
        backstory="""你是一名专业的数据分析师，善于从复杂的信息中提取关键洞察。
        你能够识别模式、趋势和相关性，为决策提供有价值的分析。
        你的分析总是客观、基于数据，并且能够用清晰的方式表达复杂的概念。""",
        verbose=True,
        allow_delegation=False
    )
    
    # 撰写员Agent：负责写报告
    writer_agent = Agent(
        role='技术写作专家',
        goal='将研究和分析结果转化为清晰、易懂的报告',
        backstory="""你是一名经验丰富的技术写作专家，能够将复杂的技术概念
        转化为普通人也能理解的内容。你的写作风格清晰、结构良好，
        总能抓住读者的注意力并传达关键信息。""",
        verbose=True,
        allow_delegation=False
    )
    
    print("✅ 创建了3个专业Agent：")
    print(f"  🔬 {researcher_agent.role}")
    print(f"  📊 {analyst_agent.role}")
    print(f"  ✍️ {writer_agent.role}")
    
    return researcher_agent, analyst_agent, writer_agent

# 创建Agent团队
researcher, analyst, writer = create_research_crew()

✅ 创建了3个专业Agent：
  🔬 高级研究员
  📊 数据分析师
  ✍️ 技术写作专家


In [8]:
# 创建Task（任务）
def create_research_tasks(topic: str):
    """
    为研究团队创建具体的任务
    
    参数:
        topic: 研究主题
    
    返回:
        任务列表
    """
    
    # 任务1：研究
    research_task = Task(
        description=f"""
        深入研究主题：{topic}
        
        具体要求：
        1. 收集关于该主题的最新信息
        2. 分析当前的发展状况
        3. 识别关键趋势和挑战
        4. 收集相关的统计数据和案例
        
        输出：详细的研究报告，包含事实、数据和观察结果
        """,
        agent=researcher,  # 指定执行这个任务的Agent
        expected_output="包含事实、数据和趋势分析的详细研究报告"
    )
    
    # 任务2：分析
    analysis_task = Task(
        description=f"""
        基于研究员的报告，对{topic}进行深入分析
        
        具体要求：
        1. 分析关键趋势和模式
        2. 评估机会和风险
        3. 提供数据驱动的洞察
        4. 预测未来发展方向
        
        输出：结构化的分析报告，包含洞察和建议
        """,
        agent=analyst,
        expected_output="包含洞察、建议和预测的分析报告"
    )
    
    # 任务3：撰写最终报告
    writing_task = Task(
        description=f"""
        基于研究和分析结果，撰写关于{topic}的综合报告
        
        具体要求：
        1. 整合研究和分析的所有发现
        2. 使用清晰、易懂的语言
        3. 提供实用的结论和建议
        4. 确保报告结构良好、逻辑清晰
        
        输出：完整的综合报告，适合非专业读者阅读
        """,
        agent=writer,
        expected_output="结构清晰、易懂的综合研究报告"
    )
    
    print("✅ 创建了3个相关任务：")
    print("  📋 研究任务 → 收集信息")
    print("  📋 分析任务 → 深入分析")
    print("  📋 撰写任务 → 综合报告")
    
    return [research_task, analysis_task, writing_task]

# 创建任务
topic = "大语言模型在教育领域的应用前景"
tasks = create_research_tasks(topic)

print(f"\n🎯 研究主题：{topic}")

✅ 创建了3个相关任务：
  📋 研究任务 → 收集信息
  📋 分析任务 → 深入分析
  📋 撰写任务 → 综合报告

🎯 研究主题：大语言模型在教育领域的应用前景


In [12]:
from crewai import Agent, Task, Crew

# 创建Crew（团队）并执行任务
def execute_research_project(agents_list, tasks_list):
    """
    执行完整的研究项目
    
    参数:
        agents_list: Agent列表
        tasks_list: 任务列表
    
    返回:
        执行结果
    """
    
    # 创建Crew（团队）
    research_crew = Crew(
        agents=agents_list,  # 团队成员
        tasks=tasks_list,    # 任务列表
        verbose=True,           # 详细程度（0-2）
        process='sequential' # 执行模式：sequential（顺序）或 hierarchical（层级）
    )
    
    print("\n🚀 开始执行研究项目...")
    print("团队将按以下顺序工作：")
    print("  1️⃣ 研究员收集信息")
    print("  2️⃣ 分析师分析数据")
    print("  3️⃣ 撰写员写综合报告")
    print("\n" + "="*60)
    
    # 执行任务
    # kickoff()方法启动整个工作流程
    result = research_crew.kickoff()
    
    return result

# 执行项目（这可能需要几分钟）
try:
    final_result = execute_research_project(
        [researcher, analyst, writer], 
        tasks
    )
    
    print("\n" + "="*60)
    print("🎉 项目执行完成！")
    print("\n📄 最终报告：")
    print("-"*60)
    print(final_result)
    
except Exception as e:
    print(f"\n❌ 执行过程中出现错误: {e}")
    print("\n💡 这可能是因为：")
    print("  - API配额限制")
    print("  - 网络连接问题")
    print("  - CrewAI版本兼容性")
    print("\n🔧 解决方案：")
    print("  - 检查API key配置")
    print("  - 稍后重试")
    print("  - 使用简化版本的演示")


🚀 开始执行研究项目...
团队将按以下顺序工作：
  1️⃣ 研究员收集信息
  2️⃣ 分析师分析数据
  3️⃣ 撰写员写综合报告



╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 23e2982f-eca1-4b85-959b-0721b1e75b59                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: 高级研究员                                                                                              │
│                                                                                                                 │
│  Task:                                                                                                          │
│          深入研究主题：大语言模型在教育领域的应用前景                                                           │
│                                                                                                                 │
│          具体要求：                                                                                             │
│          1. 收集关于该主题的最新信息                                                                            │
│          2. 分析当前的发展状况                                                                                  │
│          3. 识别关键趋势和挑战                                                                                  │
│          4. 收集相关的统计数据和案例                                                                            │
│                                                                                                                 │
│          输出：详细的研究报告，包含事实、数据和观察结果                                                         │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: 高级研究员                                                                                              │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  ### 研究报告：大语言模型在教育领域的应用前景                                                                   │
│                                                                                                                 │
│  #### 1. 引言                                                                                                   │
│  大语言模型（Large Language Models,                                                                             │
│  LLMs）是近年来人工智能领域的重要发展，其在教育领域的应用潜力引起了广泛关注。本文将深入探讨大语言模型在教育中   │
│  的具体应用、当前的发展状况、关键趋势、面临的挑战以及相关的统计数据和案例。                                     │
│                                                                                                                 │
│  #### 2. 当前发展状况                                                                                           │
│  近年来，随着计算能力的增加和自然语言处理（NLP）技术的进步，大语言模型如GPT-3和BERT等在教育领域中已经取得了显   │
│  著进展。这些模型能够通过生成自然语言文本、理解上下文、进行对话等功能来辅助学习和教学。                         │
│                                                                                                                 │
│  - **教学辅导**：利用LLMs进行个性化学习辅导，解决学生在理解课程内容中的难题。比如，教育平台Khan                 │
│  Academy正在探索使用大语言模型为学生提供实时的数学问题解决指导。                                                │
│  -                                                                                                              │
│  **语言学习**：LLMs能帮助学生练习语言技能，通过对话模拟与评价提供即时反馈。例如，Duolingo等语言学习应用已开始   │
│  整合类似技术以提升学习体验。                                                                                   │
│                                                                                                                 │
│  #### 3. 关键趋势                                                                                               │
│  -                                                                                                              │
│  **个性化学习**：LLMs的一个重要趋势是推动个性化教育，通过分析学生数据为每个学生定制适合其学习风格的内容和材料   │
│  。                                                                                                             │
│  - **对话代理的使用增加**：通过与学生进行互动，LLMs能够提供即时反馈和解答问题，从而增强学习参与感。             │
│  - **内容生成与评估**：教师利用LLMs生成试题、教育资料，甚至评估学生的作业，提高工作效率。                       │
│                                                                                                                 │
│  #### 4. 挑战                                                                                                   │
│  尽管大语言模型在教育领域展现出强大的应用潜力，但在其应用过程中仍存在若干挑战：                                 │
│  - **数据隐私与安全性**：如何确保学生个人数据的安全性是教育机构必须面对的重要问题。                             │
│  -                                                                                                              │
│  **准确性与偏见问题**：LLMs可能生成偏见内容或错误信息，影响学生的学习质量。因此，对模型输出进行有效监控和调节   │
│  显得尤为重要。                                                                                                 │
│  - **教师的接受度与教育培训**：教师对新技术的接受度以及相应的培训需求将直接影响LLMs的有效应用。                 │
│                                                                                                                 │
│  #### 5. 统计数据与案例                                                                                         │
│  根据2023年的教育技术观察报告，已有超过40%的教育机构开始试验使用大语言模型。以下是一些具体的统计数据：          │
│  - 超过42%的学生表示他们希望在学习过程中使用人工智能工具。                                                      │
│  - 98%的教师表示他们认为LLMs有助于提升学生学习兴趣。                                                            │
│        

...

Output()

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: 技术写作专家                                                                                            │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  ### 研究报告：大语言模型在教育领域的应用前景                                                                   │
│                                                                                                                 │
│  #### 1. 引言                                                                                                   │
│  随着人工智能的飞速发展，特别是在自然语言处理技术的进步背景下，大语言模型（Large Language Models,               │
│  LLMs）的应用在教育领域展现出广阔的前景。本文旨在整合现有的研究与分析结果，探讨大语言模型在教育中的潜在应用、   │
│  当前发展状况以及未来的趋势，同时提出相关的结论和建议，以帮助教育工作者与政策制定者更好地利用这一先进技术。     │
│                                                                                                                 │
│  #### 2. 当前发展状况                                                                                           │
│  近年来，大语言模型在教育领域的应用正在快速增长，主要体现在以下几个方面：                                       │
│                                                                                                                 │
│  - **教学辅导**：LLMs通过提供个性化的学习体验，极大地帮助学生克服学习中的困难。例如，Khan Academy               │
│  正在探索基于大语言模型的实时数学问题解决指导，为学生提供个性化的支持。                                         │
│                                                                                                                 │
│  -                                                                                                              │
│  **语言学习**：语言学习应用（如Duolingo）开始整合LLMs，通过对话模拟和即时反馈来帮助学生练习语言，从而显著提升   │
│  学习效果。                                                                                                     │
│                                                                                                                 │
│  #### 3. 关键趋势                                                                                               │
│  1. **个性化学习**：LLMs通过分析学生的学习数据，能够为学生提供定制化的学习内容和材料，推动个性化教育的发展。    │
│                                                                                                                 │
│  2. **对话代理的使用增加**：LLMs可以与学生进行互动，提供即时的解答和反馈，从而增强学习的参与感和主动性。        │
│                                                                                                                 │
│  3.                                                                                                             │
│  **内容生成与评估**：教师能够利用LLMs快速生成测验、教学材料和评估工具，提高教学效率，减轻日常准备工作的压力。   │
│                                                                                                                 │
│  #### 4. 挑战                                                                                                   │
│  尽管大语言模型具有显著的优势，但在实际应用中仍面临以下挑战：                                                   │
│                                                                                                                 │
│  - **数据隐私与安全性**：确保学生的个人信息安全是教育机构必须面对的重要问题，需制定相应的隐私政策。             │
│                                                                                                                 │
│  -                                                                                                              │
│  **准确性与偏见问题**：LLMs可能生成带有偏见或错误的信息，这对学生的学习质量造成影响，因此需要对模型输出进行严   │
│  格监控。                                                                                                       │
│                                                                                                                 │
│  - **教师的接受度与教育培训**：教师对新技术的接受程度及必要的培训，将直接关系到LLMs在教育中的成功实施效果。     │
│                       

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 143d3b93-9660-4519-9561-c1f2d1754290                                                                     │
│  Agent: 技术写作专家                                                                                            │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: 23e2982f-eca1-4b85-959b-0721b1e75b59                                                                       │
│  Tool Args:                                                                                                     │
│  Final Output: ### 研究报告：大语言模型在教育领域的应用前景                                                     │
│                                                                                                                 │
│  #### 1. 引言                                                                                                   │
│  随着人工智能的飞速发展，特别是在自然语言处理技术的进步背景下，大语言模型（Large Language Models,               │
│  LLMs）的应用在教育领域展现出广阔的前景。本文旨在整合现有的研究与分析结果，探讨大语言模型在教育中的潜在应用、   │
│  当前发展状况以及未来的趋势，同时提出相关的结论和建议，以帮助教育工作者与政策制定者更好地利用这一先进技术。     │
│                                                                                                                 │
│  #### 2. 当前发展状况                                                                                           │
│  近年来，大语言模型在教育领域的应用正在快速增长，主要体现在以下几个方面：                                       │
│                                                                                                                 │
│  - **教学辅导**：LLMs通过提供个性化的学习体验，极大地帮助学生克服学习中的困难。例如，Khan Academy               │
│  正在探索基于大语言模型的实时数学问题解决指导，为学生提供个性化的支持。                                         │
│                                                                                                                 │
│  -                                                                                                              │
│  **语言学习**：语言学习应用（如Duolingo）开始整合LLMs，通过对话模拟和即时反馈来帮助学生练习语言，从而显著提升   │
│  学习效果。                                                                                                     │
│                                                                                                                 │
│  #### 3. 关键趋势                                                                                               │
│  1. **个性化学习**：LLMs通过分析学生的学习数据，能够为学生提供定制化的学习内容和材料，推动个性化教育的发展。    │
│                                                                                                                 │
│  2. **对话代理的使用增加**：LLMs可以与学生进行互动，提供即时的解答和反馈，从而增强学习的参与感和主动性。        │
│                                                                                                                 │
│  3.                                                                                                             │
│  **内容生成与评估**：教师能够利用LLMs快速生成测验、教学材料和评估工具，提高教学效率，减轻日常准备工作的压力。   │
│                                                                                                                 │
│  #### 4. 挑战                                                                                                   │
│  尽管大语言模型具有显著的优势，但在实际应用中仍面临以下挑战：                                                   │
│                                                                                                                 │
│  - **数据隐私与安全性**：确保学生的个人信息安全是教育机构必须面对的重要问题，需制定相应的隐私政策。             │
│                                                                                                                 │
│  -                                                                                                              │
│  **准确性与偏见问题**：LLMs可能生成带有偏见或错误的信息，这对学生的学习质量造成影响，因此需要对模型输出进行严   │
│  格监控。                                                                                                       │
│                                                                                    


🎉 项目执行完成！

📄 最终报告：
------------------------------------------------------------
### 研究报告：大语言模型在教育领域的应用前景

#### 1. 引言
随着人工智能的飞速发展，特别是在自然语言处理技术的进步背景下，大语言模型（Large Language Models, LLMs）的应用在教育领域展现出广阔的前景。本文旨在整合现有的研究与分析结果，探讨大语言模型在教育中的潜在应用、当前发展状况以及未来的趋势，同时提出相关的结论和建议，以帮助教育工作者与政策制定者更好地利用这一先进技术。

#### 2. 当前发展状况
近年来，大语言模型在教育领域的应用正在快速增长，主要体现在以下几个方面：

- **教学辅导**：LLMs通过提供个性化的学习体验，极大地帮助学生克服学习中的困难。例如，Khan Academy 正在探索基于大语言模型的实时数学问题解决指导，为学生提供个性化的支持。
  
- **语言学习**：语言学习应用（如Duolingo）开始整合LLMs，通过对话模拟和即时反馈来帮助学生练习语言，从而显著提升学习效果。

#### 3. 关键趋势
1. **个性化学习**：LLMs通过分析学生的学习数据，能够为学生提供定制化的学习内容和材料，推动个性化教育的发展。
   
2. **对话代理的使用增加**：LLMs可以与学生进行互动，提供即时的解答和反馈，从而增强学习的参与感和主动性。

3. **内容生成与评估**：教师能够利用LLMs快速生成测验、教学材料和评估工具，提高教学效率，减轻日常准备工作的压力。

#### 4. 挑战
尽管大语言模型具有显著的优势，但在实际应用中仍面临以下挑战：

- **数据隐私与安全性**：确保学生的个人信息安全是教育机构必须面对的重要问题，需制定相应的隐私政策。

- **准确性与偏见问题**：LLMs可能生成带有偏见或错误的信息，这对学生的学习质量造成影响，因此需要对模型输出进行严格监控。

- **教师的接受度与教育培训**：教师对新技术的接受程度及必要的培训，将直接关系到LLMs在教育中的成功实施效果。

#### 5. 统计数据与案例
根据2023年的教育技术观察报告，已有超过40%的教育机构开始试点使用大语言模型。具体数据如下：
-

## 4. 深入理解Agent协作模式

### 4.1 四种主要架构模式

让我们用代码演示这四种架构：

In [13]:
from typing import List, Dict
import random

class ArchitectureDemo:
    """演示不同的多Agent架构模式"""
    
    def __init__(self):
        # 创建一组基础Agent
        self.agents = {
            "产品经理": SimpleAgent("产品经理Alice", "产品经理", "战略思维，用户导向"),
            "开发工程师": SimpleAgent("开发工程师Bob", "开发工程师", "技术专业，注重实现"),
            "测试工程师": SimpleAgent("测试工程师Carol", "测试工程师", "细致严谨，质量保证"),
            "设计师": SimpleAgent("设计师David", "UI设计师", "创意思维，用户体验")
        }
    
    def supervisor_architecture(self, task: str):
        """
        1. Supervisor架构（监督者模式）
        有一个中央Agent负责任务分配和协调
        """
        print("\n🏢 Supervisor架构演示")
        print("特点：中央协调，分层管理")
        
        # 创建Supervisor Agent
        supervisor = SimpleAgent(
            "项目经理Eve", 
            "项目经理", 
            "统筹规划，协调各方，确保项目顺利进行"
        )
        
        # Supervisor分配任务
        task_allocation = supervisor.respond(
            f"请为以下项目任务分配给合适的团队成员：{task}。团队有：产品经理、开发工程师、测试工程师、设计师。"
        )
        print(f"\n👔 {supervisor.name}的任务分配：")
        print(task_allocation)
        
        # 模拟各Agent执行分配的子任务
        results = {}
        for role, agent in list(self.agents.items())[:2]:  # 演示2个Agent
            subtask = f"从{role}角度处理：{task}"
            result = agent.respond(subtask)
            results[role] = result
            print(f"\n{agent.name}完成工作：{result[:100]}...")
        
        return results
    
    def network_architecture(self, task: str):
        """
        2. Network架构（网络模式）
        Agent之间可以自由通信和协作
        """
        print("\n🕸️ Network架构演示")
        print("特点：自由通信，去中心化协作")
        
        # 随机选择两个Agent进行对话
        selected_agents = random.sample(list(self.agents.items()), 2)
        agent1_role, agent1 = selected_agents[0]
        agent2_role, agent2 = selected_agents[1]
        
        print(f"\n💬 {agent1.name} 和 {agent2.name} 开始协作")
        
        # Agent1先发起
        message1 = agent1.respond(f"我们来合作处理：{task}。作为{agent1_role}，我认为...")
        print(f"\n{agent1.name}: {message1[:120]}...")
        
        # Agent2回应
        message2 = agent2.respond(f"基于{agent1_role}的观点：{message1[:200]}... 作为{agent2_role}，我补充...")
        print(f"\n{agent2.name}: {message2[:120]}...")
        
        return {agent1_role: message1, agent2_role: message2}
    
    def hierarchical_architecture(self, task: str):
        """
        3. Hierarchical架构（层级模式）
        多层管理结构，每层有不同的职责
        """
        print("\n🏗️ Hierarchical架构演示")
        print("特点：分层管理，逐级执行")
        
        # 高层：战略决策
        ceo = SimpleAgent("CEO Frank", "首席执行官", "战略视野，决策果断")
        strategic_decision = ceo.respond(f"从公司战略角度，如何处理：{task}")
        print(f"\n🎯 高层决策（{ceo.name}）：")
        print(strategic_decision[:150] + "...")
        
        # 中层：具体规划
        pm = self.agents["产品经理"]
        tactical_plan = pm.respond(f"基于高层决策：{strategic_decision[:100]}...，制定具体执行计划")
        print(f"\n📋 中层规划（{pm.name}）：")
        print(tactical_plan[:150] + "...")
        
        # 基层：执行实施
        dev = self.agents["开发工程师"]
        implementation = dev.respond(f"基于规划：{tactical_plan[:100]}...，开始技术实现")
        print(f"\n⚙️ 基层执行（{dev.name}）：")
        print(implementation[:150] + "...")
        
        return {
            "strategic": strategic_decision,
            "tactical": tactical_plan,
            "operational": implementation
        }
    
    def swarm_architecture(self, task: str):
        """
        4. Swarm架构（群体模式）
        多个Agent协同工作，动态调整
        """
        print("\n🐝 Swarm架构演示")
        print("特点：群体智慧，自适应协作")
        
        # 所有Agent同时对任务提出初步想法
        initial_ideas = {}
        for role, agent in self.agents.items():
            idea = agent.respond(f"快速提出关于'{task}'的一个创意想法")
            initial_ideas[role] = idea
            print(f"\n💡 {agent.name}的想法：{idea[:80]}...")
        
        # 群体投票选择最佳想法（模拟）
        best_role = random.choice(list(initial_ideas.keys()))
        print(f"\n🗳️ 群体投票选择了 {best_role} 的想法")
        
        # 其他Agent基于最佳想法进行改进
        improvements = {}
        for role, agent in list(self.agents.items())[:2]:  # 演示2个Agent
            if role != best_role:
                improvement = agent.respond(
                    f"基于{best_role}的想法：{initial_ideas[best_role][:100]}...，提出改进建议"
                )
                improvements[role] = improvement
                print(f"\n🔧 {agent.name}的改进：{improvement[:80]}...")
        
        return {
            "original_ideas": initial_ideas,
            "selected": initial_ideas[best_role],
            "improvements": improvements
        }

# 演示所有架构模式
demo = ArchitectureDemo()
task = "开发一个AI驱动的在线学习平台"

print(f"🎯 演示任务：{task}")
print("\n我们将用4种不同的架构来处理这个任务...")

# 依次演示每种架构
architectures = [
    ("Supervisor", demo.supervisor_architecture),
    ("Network", demo.network_architecture),
    ("Hierarchical", demo.hierarchical_architecture),
    ("Swarm", demo.swarm_architecture)
]

for arch_name, arch_method in architectures[:2]:  # 演示前两种
    try:
        result = arch_method(task)
        print(f"\n✅ {arch_name}架构演示完成")
        print("-" * 60)
    except Exception as e:
        print(f"\n❌ {arch_name}架构演示出错: {e}")

🎯 演示任务：开发一个AI驱动的在线学习平台

我们将用4种不同的架构来处理这个任务...

🏢 Supervisor架构演示
特点：中央协调，分层管理

👔 项目经理Eve的任务分配：
好的，针对“开发一个AI驱动的在线学习平台”项目，我建议如下分工：

1. 产品经理：负责梳理项目需求，确定产品功能，制定产品路线图，并与各方沟通协调，确保产品方向与市场需求一致。

2. 开发工程师：负责AI功能的研发、平台架构的搭建及核心技术实现。包括后端、前端以及与AI模型的集成开发。

3. 测试工程师：负责制定测试计划，设计测试用例，对平台的各项功能和AI模块进行全面测试，确保产品质量和稳定性。

4. 设计师：负责平台的UI/UX设计，保证用户界面美观、易用，并与产品经理和开发工程师密切协作，确保设计方案的可实现性。

我会持续跟进各环节进展，及时协调资源与沟通，确保项目按计划推进。如有疑问或需资源支持，请随时反馈。

产品经理Alice完成工作：当然可以！作为产品经理Alice，我会从战略和用户需求两个层面入手，梳理AI驱动在线学习平台的开发思路：

**一、战略层面**

1. **市场定位**：首先要明确我们的目标用户是谁（如K12学生、...

开发工程师Bob完成工作：好的，这里是作为开发工程师Bob的思路和实现建议：

1. 技术选型  
- 前端：React（或Next.js），保证交互和响应速度。  
- 后端：Node.js（Express/Koa），利于A...

✅ Supervisor架构演示完成
------------------------------------------------------------

🕸️ Network架构演示
特点：自由通信，去中心化协作

💬 产品经理Alice 和 测试工程师Carol 开始协作

产品经理Alice: 太好了，很高兴和你合作开发AI驱动的在线学习平台！

作为产品经理，我认为我们首先要明确两点：一是目标用户是谁，他们的核心需求是什么；二是AI技术如何切实提升学习体验，而不仅仅是“炫技”。

我的初步思考是——

1. **用户画像与需求*...

测试工程师Carol: 非常感谢您的梳理和方向引导！作为测试工程师，我从质量保证和风险把控的角度补充几点建议，帮助我们更系统地推进设计和开发

## 5. 实战：构建内容创作团队

让我们创建一个更实用的例子：一个内容创作团队。

In [14]:
class ContentCreationTeam:
    """
    内容创作团队
    演示一个完整的多Agent协作流程
    """
    
    def __init__(self):
        # 创建专业的内容创作团队
        self.researcher = Agent(
            role='内容研究专员',
            goal='深入研究主题，收集最新最准确的信息',
            backstory="""你是一名专业的内容研究员，擅长快速收集和整理信息。
            你总能找到最新的趋势、数据和案例，为内容创作提供坚实的基础。""",
            verbose=True,
            allow_delegation=False
        )
        
        self.copywriter = Agent(
            role='创意文案',
            goal='创作吸引人的内容，确保信息传达清晰有效',
            backstory="""你是一名富有创意的文案，善于用生动有趣的方式表达复杂概念。
            你的文字总能抓住读者注意力，并有效传达核心信息。""",
            verbose=True,
            allow_delegation=False
        )
        
        self.editor = Agent(
            role='内容编辑',
            goal='审核和优化内容，确保质量和一致性',
            backstory="""你是一名经验丰富的编辑，有着敏锐的语言感觉和严格的质量标准。
            你擅长发现问题、优化结构，确保内容达到发布标准。""",
            verbose=True,
            allow_delegation=False
        )
        
        self.seo_specialist = Agent(
            role='SEO优化专员',
            goal='优化内容的搜索引擎表现，提高可见性',
            backstory="""你是SEO专家，深入了解搜索引擎算法和用户搜索行为。
            你能够在保持内容质量的同时，优化其搜索排名表现。""",
            verbose=True,
            allow_delegation=False
        )
    
    def create_content_workflow(self, topic: str, content_type: str = "blog"):
        """
        创建完整的内容创作工作流
        
        参数:
            topic: 内容主题
            content_type: 内容类型（blog/social/newsletter等）
        """
        
        # 任务1：研究阶段
        research_task = Task(
            description=f"""
            深入研究主题：{topic}
            
            任务要求：
            1. 收集关于{topic}的最新信息和趋势
            2. 找出目标受众最关心的问题
            3. 分析竞品内容，找出差异化角度
            4. 整理关键数据、案例和引用来源
            
            输出格式：结构化的研究报告，包含要点总结和建议角度
            """,
            agent=self.researcher,
            expected_output="详细的研究报告，包含数据、趋势和内容角度建议"
        )
        
        # 任务2：内容创作
        writing_task = Task(
            description=f"""
            基于研究报告，创作关于{topic}的{content_type}内容
            
            任务要求：
            1. 使用研究报告中的信息和角度
            2. 确保内容引人入胜、易于理解
            3. 包含实用的建议或洞察
            4. 适合目标受众的阅读习惯
            
            输出格式：完整的{content_type}内容，包含标题、正文和结尾
            """,
            agent=self.copywriter,
            expected_output=f"高质量的{content_type}内容，结构清晰，引人入胜"
        )
        
        # 任务3：内容编辑
        editing_task = Task(
            description=f"""
            审核和优化{content_type}内容
            
            任务要求：
            1. 检查语法、拼写和表达
            2. 优化内容结构和逻辑流程
            3. 确保信息准确性和一致性
            4. 提升内容的可读性和影响力
            
            输出格式：经过编辑优化的最终内容
            """,
            agent=self.editor,
            expected_output="经过专业编辑的高质量内容"
        )
        
        # 任务4：SEO优化
        seo_task = Task(
            description=f"""
            对内容进行SEO优化
            
            任务要求：
            1. 优化标题和子标题的关键词
            2. 改进内容的搜索引擎友好性
            3. 建议Meta描述和标签
            4. 确保内容长度和关键词密度合适
            
            输出格式：SEO优化后的内容 + 优化建议报告
            """,
            agent=self.seo_specialist,
            expected_output="SEO优化的内容和具体的优化建议"
        )
        
        # 创建内容创作团队
        content_crew = Crew(
            agents=[self.researcher, self.copywriter, self.editor, self.seo_specialist],
            tasks=[research_task, writing_task, editing_task, seo_task],
            verbose=2,
            process='sequential'  # 按顺序执行
        )
        
        return content_crew
    
    def create_content(self, topic: str, content_type: str = "blog"):
        """
        执行完整的内容创作流程
        """
        print(f"\n🎬 开始内容创作：{topic}")
        print(f"内容类型：{content_type}")
        print("\n团队协作流程：")
        print("  1️⃣ 研究专员 → 收集信息")
        print("  2️⃣ 创意文案 → 撰写内容")
        print("  3️⃣ 内容编辑 → 审核优化")
        print("  4️⃣ SEO专员 → 搜索优化")
        
        try:
            crew = self.create_content_workflow(topic, content_type)
            result = crew.kickoff()
            return result
        except Exception as e:
            print(f"\n❌ 创作过程出错: {e}")
            return None

# 创建内容创作团队
content_team = ContentCreationTeam()

print("\n🎨 内容创作团队已组建")
print("团队成员：研究专员、创意文案、内容编辑、SEO专员")
print("\n这个团队展示了专业分工和协作的优势！")


🎨 内容创作团队已组建
团队成员：研究专员、创意文案、内容编辑、SEO专员

这个团队展示了专业分工和协作的优势！


In [15]:
# 执行一个小规模的内容创作任务
print("\n🧪 执行内容创作演示...")

# 为了演示，我们创建一个简化版本
def simplified_content_creation(topic: str):
    """
    简化版的内容创作演示
    适用于API限制的情况
    """
    print(f"\n📝 简化演示：{topic}")
    
    # 使用我们之前创建的SimpleAgent
    researcher = SimpleAgent("研究员", "内容研究员", "善于收集信息，分析趋势")
    writer = SimpleAgent("文案", "创意文案", "善于创作，表达生动")
    
    # 研究阶段
    research = researcher.respond(f"研究主题：{topic}，总结3个关键要点")
    print(f"\n🔍 研究结果：\n{research[:200]}...")
    
    # 创作阶段
    content = writer.respond(f"基于研究：{research}，写一篇简短的文章")
    print(f"\n✍️ 创作内容：\n{content[:300]}...")
    
    return {"research": research, "content": content}

# 执行简化演示
demo_topic = "远程工作的未来趋势"
demo_result = simplified_content_creation(demo_topic)

print("\n✅ 演示完成！")
print("\n💡 这展示了多Agent协作的基本原理：")
print("  - 专业分工：每个Agent有自己的专长")
print("  - 顺序协作：前一个Agent的输出是后一个的输入")
print("  - 质量提升：多个Agent的协作提高了最终质量")


🧪 执行内容创作演示...

📝 简化演示：远程工作的未来趋势

🔍 研究结果：
作为研究员，针对“远程工作的未来趋势”，我总结了以下三个关键要点：

1. 混合办公模式将成为主流  
调研数据显示，越来越多企业倾向于采用“远程+现场”结合的混合办公模式。这种模式能够兼顾员工灵活性和团队协作效率，预计未来三到五年将成为大部分知识型企业的常态。

2. 技术工具推动协作创新  
随着云计算、视频会议、协作平台等技术的不断进步，远程工作的协作效率和安全性大幅提升。未来，AI 助手、...

✍️ 创作内容：
远程工作的未来，正悄然加速。

一边是“混合办公”成为新常态——既能让员工在家高效安静地完成任务，又能在办公室碰撞创意的火花。灵活与协作，在混合模式下找到了最佳平衡点。

一边是技术工具飞速进化。云端存储、视频会议、在线协作平台，已经重塑了我们的工作方式。更令人期待的是，AI助手和虚拟现实正跃跃欲试，准备带来前所未有的远程协作体验。数字化办公，从未如此智能与便捷。

但远程工作的路上，也有挑战：如何评估绩效？怎样让员工有归属感？企业开始转向结果导向管理，线上文化、心理健康支持、职业成长……这些新的关注点，正成为企业凝聚人心的新法宝。

远程办公的未来，是技术驱动的创新，是人性化的管理，是灵活与...

✅ 演示完成！

💡 这展示了多Agent协作的基本原理：
  - 专业分工：每个Agent有自己的专长
  - 顺序协作：前一个Agent的输出是后一个的输入
  - 质量提升：多个Agent的协作提高了最终质量


## 6. 多Agent系统的高级话题

### 6.1 通信协议

在复杂的多Agent系统中，Agent之间需要有效的通信机制：

In [16]:
from typing import Dict, List, Any
from dataclasses import dataclass
from datetime import datetime

@dataclass
class Message:
    """Agent间的消息格式"""
    sender: str          # 发送者
    receiver: str        # 接收者
    content: str         # 消息内容
    message_type: str    # 消息类型：request/response/notification
    timestamp: datetime  # 时间戳
    metadata: Dict[str, Any] = None  # 额外元数据

class MessageBus:
    """消息总线：管理Agent间的通信"""
    
    def __init__(self):
        self.messages: List[Message] = []  # 消息历史
        self.agents: Dict[str, Any] = {}   # 注册的Agent
    
    def register_agent(self, agent_name: str, agent: Any):
        """注册Agent到消息总线"""
        self.agents[agent_name] = agent
        print(f"📝 Agent '{agent_name}' 已注册到消息总线")
    
    def send_message(self, sender: str, receiver: str, content: str, 
                    message_type: str = "request", metadata: Dict = None):
        """发送消息"""
        message = Message(
            sender=sender,
            receiver=receiver,
            content=content,
            message_type=message_type,
            timestamp=datetime.now(),
            metadata=metadata or {}
        )
        
        self.messages.append(message)
        print(f"📤 {sender} → {receiver}: {content[:50]}...")
        
        return message
    
    def get_messages_for_agent(self, agent_name: str) -> List[Message]:
        """获取指定Agent的消息"""
        return [msg for msg in self.messages if msg.receiver == agent_name]
    
    def get_conversation(self, agent1: str, agent2: str) -> List[Message]:
        """获取两个Agent之间的对话"""
        return [
            msg for msg in self.messages 
            if (msg.sender == agent1 and msg.receiver == agent2) or 
               (msg.sender == agent2 and msg.receiver == agent1)
        ]

# 演示消息总线
message_bus = MessageBus()

# 注册一些Agent
agent_names = ["产品经理", "开发工程师", "测试工程师"]
for name in agent_names:
    message_bus.register_agent(name, None)  # 简化演示

# 模拟消息传递
print("\n💬 模拟Agent间的消息传递：")

message_bus.send_message(
    "产品经理", "开发工程师", 
    "请评估新功能的技术可行性", 
    "request"
)

message_bus.send_message(
    "开发工程师", "产品经理", 
    "技术上可行，预计需要2周开发时间", 
    "response"
)

message_bus.send_message(
    "产品经理", "测试工程师", 
    "请准备新功能的测试计划", 
    "request"
)

# 查看对话历史
conversation = message_bus.get_conversation("产品经理", "开发工程师")
print(f"\n📜 产品经理和开发工程师的对话历史：")
for msg in conversation:
    print(f"  {msg.timestamp.strftime('%H:%M')} | {msg.sender}: {msg.content}")

print("\n💡 消息总线的优势：")
print("  - 解耦：Agent不需要直接引用其他Agent")
print("  - 可追溯：所有通信都有记录")
print("  - 可扩展：容易添加新的Agent")
print("  - 可控制：可以添加权限、过滤等功能")

📝 Agent '产品经理' 已注册到消息总线
📝 Agent '开发工程师' 已注册到消息总线
📝 Agent '测试工程师' 已注册到消息总线

💬 模拟Agent间的消息传递：
📤 产品经理 → 开发工程师: 请评估新功能的技术可行性...
📤 开发工程师 → 产品经理: 技术上可行，预计需要2周开发时间...
📤 产品经理 → 测试工程师: 请准备新功能的测试计划...

📜 产品经理和开发工程师的对话历史：
  15:55 | 产品经理: 请评估新功能的技术可行性
  15:55 | 开发工程师: 技术上可行，预计需要2周开发时间

💡 消息总线的优势：
  - 解耦：Agent不需要直接引用其他Agent
  - 可追溯：所有通信都有记录
  - 可扩展：容易添加新的Agent
  - 可控制：可以添加权限、过滤等功能


### 6.2 协作策略和冲突解决

In [18]:
class CollaborationManager:
    """
    协作管理器
    处理Agent间的协作策略和冲突解决
    """
    
    def __init__(self):
        self.agents = {}
        self.conflicts = []  # 冲突记录
    
    def register_agent(self, name: str, agent: SimpleAgent, expertise_score: float):
        """注册Agent及其专业度评分"""
        self.agents[name] = {
            "agent": agent,
            "expertise": expertise_score,
            "contribution_count": 0
        }
    
    def voting_strategy(self, question: str, options: List[str]) -> str:
        """
        投票策略：让所有Agent投票选择最佳方案
        """
        print(f"\n🗳️ 投票决策：{question}")
        print(f"选项：{options}")
        
        votes = {}
        
        for name, info in list(self.agents.items())[:3]:  # 演示3个Agent
            agent = info["agent"]
            prompt = f"针对问题：{question}，从以下选项中选择最佳方案：{options}。只回答选项编号。"
            
            try:
                response = agent.respond(prompt)
                # 简单解析投票（实际应该更robust）
                vote = options[0]  # 默认选择第一个
                for i, option in enumerate(options):
                    if str(i+1) in response or option.lower() in response.lower():
                        vote = option
                        break
                
                votes[name] = vote
                print(f"  {name}: {vote}")
                
            except Exception as e:
                print(f"  {name}: 投票失败 ({e})")
        
        # 统计投票结果
        vote_counts = {}
        for vote in votes.values():
            vote_counts[vote] = vote_counts.get(vote, 0) + 1
        
        winner = max(vote_counts.keys(), key=lambda x: vote_counts[x])
        print(f"\n🏆 投票结果：{winner} ({vote_counts[winner]}票)")
        
        return winner
    
    def expertise_weighted_decision(self, question: str, domain: str) -> str:
        """
        专业度加权决策：专业度高的Agent意见权重更大
        """
        print(f"\n⚖️ 专业度加权决策：{question}")
        print(f"领域：{domain}")
        
        # 找出该领域最专业的Agent
        expert_name = max(self.agents.keys(), 
                         key=lambda x: self.agents[x]["expertise"])
        expert_agent = self.agents[expert_name]["agent"]
        
        print(f"\n👑 领域专家：{expert_name} (专业度: {self.agents[expert_name]['expertise']})")
        
        # 专家做出决策
        expert_decision = expert_agent.respond(f"作为{domain}专家，请回答：{question}")
        print(f"\n🎯 专家决策：{expert_decision[:150]}...")
        
        # 其他Agent提供补充意见
        supplements = []
        for name, info in list(self.agents.items())[:2]:  # 演示2个Agent
            if name != expert_name:
                agent = info["agent"]
                supplement = agent.respond(
                    f"基于专家意见：{expert_decision[:100]}...，提供补充建议"
                )
                supplements.append(f"{name}: {supplement[:100]}...")
        
        print(f"\n💡 补充意见：")
        for supplement in supplements:
            print(f"  {supplement}")
        
        return expert_decision
    
    def conflict_resolution(self, issue: str, conflicting_views: Dict[str, str]) -> str:
        """
        冲突解决：当Agent意见不一致时的解决机制
        """
        print(f"\n⚡ 冲突解决：{issue}")
        print("冲突观点：")
        for agent, view in conflicting_views.items():
            print(f"  {agent}: {view[:100]}...")
        
        # 创建一个中立的仲裁者
        mediator = SimpleAgent(
            "仲裁者", "中立仲裁者", 
            "客观公正，善于平衡不同观点，寻找最佳解决方案"
        )
        
        # 仲裁者分析冲突
        conflict_summary = "\n".join([
            f"{agent}的观点：{view}" 
            for agent, view in conflicting_views.items()
        ])
        
        resolution = mediator.respond(
            f"以下是关于'{issue}'的不同观点：\n{conflict_summary}\n\n请提供一个平衡的解决方案。"
        )
        
        print(f"\n🤝 仲裁结果：{resolution[:200]}...")
        
        # 记录冲突
        self.conflicts.append({
            "issue": issue,
            "views": conflicting_views,
            "resolution": resolution,
            "timestamp": datetime.now()
        })
        
        return resolution

# 演示协作管理
collab_manager = CollaborationManager()

# 注册Agent
agents_info = [
    ("技术架构师", SimpleAgent("架构师", "技术架构师", "技术深度，系统设计"), 0.9),
    ("产品经理", SimpleAgent("产品", "产品经理", "用户需求，商业价值"), 0.7),
    ("项目经理", SimpleAgent("项目", "项目经理", "进度管理，资源协调"), 0.8)
]

for name, agent, expertise in agents_info:
    collab_manager.register_agent(name, agent, expertise)

print("\n👥 协作团队已组建")
print("成员：技术架构师、产品经理、项目经理")

# 演示投票决策
try:
    decision = collab_manager.voting_strategy(
        "选择前端技术栈",
        ["React", "Vue", "Angular"]
    )
    print(f"\n✅ 团队决策：{decision}")
except Exception as e:
    print(f"\n❌ 决策过程出错：{e}")

print("\n💡 多Agent协作的关键要素：")
print("  - 明确分工：每个Agent有清晰的职责")
print("  - 有效沟通：建立良好的消息传递机制")
print("  - 冲突解决：有处理意见分歧的策略")
print("  - 质量保证：通过协作提升最终输出质量")


👥 协作团队已组建
成员：技术架构师、产品经理、项目经理

🗳️ 投票决策：选择前端技术栈
选项：['React', 'Vue', 'Angular']
  技术架构师: React
  产品经理: React
  项目经理: React

🏆 投票结果：React (3票)

✅ 团队决策：React

💡 多Agent协作的关键要素：
  - 明确分工：每个Agent有清晰的职责
  - 有效沟通：建立良好的消息传递机制
  - 冲突解决：有处理意见分歧的策略
  - 质量保证：通过协作提升最终输出质量


## 7. 今日学习总结

### ✅ 核心概念掌握

1. **多Agent系统基础**
   - 为什么需要多个Agent：专业分工，提高效率
   - 基本协作模式：顺序、并行、层级、网络
   - 通信机制：直接传递、共享内存、消息总线

2. **CrewAI框架使用**
   - Agent：角色、目标、背景故事
   - Task：具体任务描述和期望输出
   - Crew：团队组织和执行管理
   - 工作流：sequential和hierarchical模式

3. **四种架构模式**
   - **Supervisor**：中央协调，适合有明确管理层级的场景
   - **Network**：自由通信，适合需要灵活协作的场景
   - **Hierarchical**：分层管理，适合复杂的组织结构
   - **Swarm**：群体智慧，适合需要集体决策的场景

4. **高级协作机制**
   - 消息总线：统一的通信管理
   - 投票决策：民主的决策机制
   - 专业度加权：基于能力的决策权重
   - 冲突解决：处理意见分歧的策略

### 🎯 关键收获

1. **专业分工的价值**：每个Agent专注于自己的强项，整体效果更好
2. **协作大于个体**：多个Agent的协作能够处理单个Agent无法胜任的复杂任务
3. **沟通是关键**：良好的通信机制是多Agent系统成功的基础
4. **冲突是常态**：需要有效的机制来处理不同Agent之间的意见分歧

### 💡 实践建议

1. **从简单开始**：先实现两个Agent的协作，再扩展到多个
2. **明确职责**：每个Agent的角色和目标要清晰定义
3. **设计通信协议**：提前规划好Agent之间如何传递信息
4. **考虑容错性**：单个Agent失败不应该导致整个系统崩溃
5. **监控和调试**：记录Agent的交互过程，便于优化

### 📝 明日预告：第6天 - MCP协议与企业集成

- 理解MCP（Model Context Protocol）的作用
- 学习如何连接外部系统和数据源
- 实现企业级的AI系统集成
- 构建可扩展的AI应用架构